In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, datasets, metrics
from sklearn.metrics import auc, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler


from imblearn.combine import *
from imblearn.over_sampling import *


## Trainset

In [2]:
df= pd.read_csv('total_df.csv')
df

,smiles,toxicity,ref,raw_MW,raw_ALOGP,raw_HBA,raw_HBD,raw_PSA,raw_ROTB,raw_AROM,...,1022,1023,MW,ALOGP,HBA,HBD,PSA,ROTB,AROM,ALERTS
0,CC1=NN=C(O1)C(=O)NC(C)(C)C2=N/C(=C(/NCC3=CC=C(...,1,DILIrank,444.423,0.98202,8,3,150.02,6,2,...,0.0,0.0,0.162797,-0.305971,0.405810,0.099910,0.346782,0.007493,0.434034,1.775999
1,C1=CC2=C(C=C1OC(F)(F)F)SC(=N2)N,1,DILIrank,234.202,2.77710,3,1,48.14,1,2,...,0.0,0.0,-0.434222,0.251281,-0.399196,-0.292517,-0.334802,-0.491163,0.434034,-0.893772
2,CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(...,1,DILIrank,1202.635,3.26900,12,5,278.80,15,0,...,0.0,0.0,2.316086,0.403983,1.049814,0.492336,1.208329,0.905074,-1.283533,-0.003848
3,CC(C1=CC(=CC=C1)OC2=CC=CC=C2)C(=O)O,1,DILIrank,242.274,3.66700,3,1,46.53,4,2,...,0.0,0.0,-0.411298,0.527535,-0.399196,-0.292517,-0.345573,-0.191969,0.434034,-0.893772
4,CC(=O)NC1=NN=C(S1)S(=O)(=O)N,1,DILIrank,222.251,-0.85610,5,2,115.04,2,1,...,0.0,0.0,-0.468162,-0.876584,-0.077194,-0.096304,0.112764,-0.391432,-0.424749,-0.003848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,c1ccc(cc1)CC2C(=O)NC(C(=O)NC(C(=O)NC(CSSCCC(=O...,0,Liew,1069.238,-4.34170,16,13,437.91,19,2,...,0.0,0.0,1.937244,-1.958630,1.693819,2.062043,2.272785,1.303999,0.434034,2.665922
1846,CC1C(C(CC(O1)OC2CCC3(C(C2)CCC4C3CC(C5(C4(CCC5C...,0,Liew,943.090,0.04230,19,9,282.21,10,0,...,0.0,0.0,1.578989,-0.597691,2.176822,1.277190,1.231142,0.406418,-1.283533,0.886075
1847,COc1cc(cc(c1OC)OC)C(=O)OC2CC3CN4CCc5c6ccccc6[n...,0,Liew,578.662,4.16250,9,1,108.55,7,3,...,0.0,0.0,0.544030,0.681355,0.566811,-0.292517,0.069345,0.107225,1.292817,-0.003848
1848,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc(c1)[N+](C)(C)C)C...,0,Liew,556.792,6.76220,4,0,59.08,15,2,...,0.0,0.0,0.481920,1.488388,-0.238195,-0.488730,-0.261613,0.905074,0.434034,0.886075


In [3]:
x = df.iloc[:,11:]
y = df['toxicity']

In [4]:
x

,0,1,2,3,4,5,6,7,8,9,...,1022,1023,MW,ALOGP,HBA,HBD,PSA,ROTB,AROM,ALERTS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.162797,-0.305971,0.405810,0.099910,0.346782,0.007493,0.434034,1.775999
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.434222,0.251281,-0.399196,-0.292517,-0.334802,-0.491163,0.434034,-0.893772
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.316086,0.403983,1.049814,0.492336,1.208329,0.905074,-1.283533,-0.003848
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.411298,0.527535,-0.399196,-0.292517,-0.345573,-0.191969,0.434034,-0.893772
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.468162,-0.876584,-0.077194,-0.096304,0.112764,-0.391432,-0.424749,-0.003848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.937244,-1.958630,1.693819,2.062043,2.272785,1.303999,0.434034,2.665922
1846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.578989,-0.597691,2.176822,1.277190,1.231142,0.406418,-1.283533,0.886075
1847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.544030,0.681355,0.566811,-0.292517,0.069345,0.107225,1.292817,-0.003848
1848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.481920,1.488388,-0.238195,-0.488730,-0.261613,0.905074,0.434034,0.886075


# StratifiedkFold

In [8]:
import joblib

model = joblib.load('random_RF.pkl')

# StratifiedKFold 10-fold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=8)

In [9]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

# model evaluation
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'specificity': make_scorer(specificity_score)
}

# cross validation
for metric, scorer in scoring.items():
    scores = cross_val_score(model, x, y, cv=cv, scoring=scorer)
    print(f'{metric} scores: {scores}')
    print(f'Mean {metric} score: {np.mean(scores)}')
    print('---')

accuracy scores: [0.79459459 0.78918919 0.75675676 0.8        0.81081081 0.77297297
 0.79459459 0.81621622 0.7027027  0.77837838]
Mean accuracy score: 0.7816216216216216
---
precision scores: [0.76851852 0.82022472 0.76595745 0.81521739 0.80612245 0.75238095
 0.78787879 0.8019802  0.71276596 0.78125   ]
Mean precision score: 0.7812296420438443
---
recall scores: [0.86458333 0.76041667 0.75789474 0.78947368 0.83157895 0.83157895
 0.82105263 0.85263158 0.70526316 0.78947368]
Mean recall score: 0.8003947368421052
---
f1 scores: [0.81372549 0.78918919 0.76190476 0.80213904 0.81865285 0.79
 0.80412371 0.82653061 0.70899471 0.78534031]
Mean f1 score: 0.7900600675180056
---
specificity scores: [0.71910112 0.82022472 0.75555556 0.81111111 0.78888889 0.71111111
 0.76666667 0.77777778 0.7        0.76666667]
Mean specificity score: 0.7617103620474407
---
